In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [9]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("data_train", {}, "../../data/models/detectron2/train/_annotations.coco.json", "../../data/models/detectron2/train")
register_coco_instances("data_val", {}, "../../data/models/detectron2/valid/_annotations.coco.json", "../../data/models/detectron2/valid")
register_coco_instances("data_test", {}, "../../data/models/detectron2/test/_annotations.coco.json", "../../data/models/detectron2/test")

In [25]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_train",)
cfg.DATASETS.TEST = ("data_val",)

cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
#cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.STEPS = []
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 # n classes + 1

cfg.TEST.EVAL_PERIOD = 300

In [13]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [18]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/15 20:15:22 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_68b088.pkl: 421MB [01:43, 4.08MB/s]                               
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box

[12/15 20:17:05 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\Gary Blackwood\dev\glasgow-litter\venv\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/15 20:17:41 d2.utils.events]:  eta: 0:27:13  iter: 19  total_loss: 3.136  loss_cls: 0.9554  loss_box_reg: 0.2783  loss_rpn_cls: 1.589  loss_rpn_loc: 0.1839  time: 1.1194  data_time: 0.1067  lr: 1.9981e-05  max_mem: 7616M
[12/15 20:18:02 d2.utils.events]:  eta: 0:26:49  iter: 39  total_loss: 1.826  loss_cls: 0.8223  loss_box_reg: 0.4991  loss_rpn_cls: 0.3997  loss_rpn_loc: 0.1078  time: 1.0803  data_time: 0.0406  lr: 3.9961e-05  max_mem: 7616M
[12/15 20:18:26 d2.utils.events]:  eta: 0:26:28  iter: 59  total_loss: 1.49  loss_cls: 0.6366  loss_box_reg: 0.624  loss_rpn_cls: 0.1376  loss_rpn_loc: 0.1262  time: 1.1205  data_time: 0.0421  lr: 5.9941e-05  max_mem: 7616M
[12/15 20:18:49 d2.utils.events]:  eta: 0:26:06  iter: 79  total_loss: 1.429  loss_cls: 0.5293  loss_box_reg: 0.6765  loss_rpn_cls: 0.09205  loss_rpn_loc: 0.1064  time: 1.1276  data_time: 0.0389  lr: 7.9921e-05  max_mem: 7616M
[12/15 20:19:13 d2.utils.events]:  eta: 0:25:53  iter: 99  total_loss: 1.424  loss_cls: 0.4263  lo

In [20]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4380), started 0:00:17 ago. (Use '!kill 4380' to kill it.)

In [23]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("data_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [12/15 21:10:30 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [12/15 21:10:30 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/15 21:10:30 d2.data.datasets.coco]: Loaded 122 images in COCO format from ../../data/models/detectron2/test/_annotations.coco.json
[12/15 21:10:30 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   litter   | 0            |   litter   | 223          |
|            |              |            |              |
|   total    | 223          |            |              |
[12/15 21:10:30 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/15

OrderedDict([('bbox',
              {'AP': 24.56876916790062,
               'AP50': 70.01000331210832,
               'AP75': 7.896313859938793,
               'APs': 24.79158871163907,
               'APm': 27.023488134910522,
               'APl': nan,
               'AP-litter': 24.56876916790062})])

In [22]:
%ls output

 Volume in drive C has no label.
 Volume Serial Number is 0AE3-82E4

 Directory of c:\Users\Gary Blackwood\dev\glasgow-litter\models\detectron2\output

15/12/2021  20:53    <DIR>          .
15/12/2021  20:53    <DIR>          ..
15/12/2021  20:54            66,049 events.out.tfevents.1639599322.Garys-PC.2768.0
15/12/2021  20:53                15 last_checkpoint
15/12/2021  20:54            42,997 metrics.json
15/12/2021  20:53       835,237,695 model_final.pth
               4 File(s)    835,346,756 bytes
               2 Dir(s)  50,929,061,888 bytes free


In [37]:
from detectron2.utils.visualizer import ColorMode
from IPython.display import Image, display
import matplotlib.pyplot as plt
import glob
import cv2

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("data_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("data_test")

for imageName in glob.glob('../../data/models/detectron2/test/*jpg')[:10]:
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], metadata=test_metadata)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  img = out.get_image()[:, :, ::-1]
  # plt.imshow(img)
  cv2.imshow("Litter", img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
